In [1]:
from langchain.agents import Tool
from langchain_openai import ChatOpenAI
# from LLM_Provider import oai_model

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
# from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated, Sequence
from langgraph.graph import Graph, END
import operator
from typing import List, Tuple



In [3]:
# 1. Define the tool function
def calculator(expression: str) -> str:
    """Just evaluates a math expression"""
    try:
        return str(eval(expression))
    except:
        return "Error in calculation"



In [30]:
# 2. Define the state structure
class AgentState(TypedDict):
    messages: Annotated[Sequence[HumanMessage | AIMessage], operator.add]
    next: str

# 3. Create the agent
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are hired to answer questions. You have access to a calculator tool.
    To use the calculator, format your response as:
    ACTION: calculator
    INPUT: <math expression>
    
    Otherwise just respond normally. In your response please make sure you respect the following wish from your employer:
    DO NOT ASK any questions or wait for a confirmation if they approve of you plan of action. Just give your best answer. 
    
    """),
    MessagesPlaceholder(variable_name="messages"),
])


In [31]:

model = ChatOpenAI()
agent = prompt | model



In [32]:
# 4. Define the agent function
# def run_agent(state: AgentState) -> AgentState:
#     messages = state["messages"]
#     response = agent.invoke({"messages": messages})
    
#     # Parse response for tool use
#     content = response.content
#     if "ACTION: calculator" in content:
#         expression = content.split("INPUT:")[1].strip()
#         result = calculator(expression)
#         # Add both the tool request and result to messages
#         new_messages = list(messages)
#         new_messages.extend([
#             AIMessage(content=content),
#             HumanMessage(content=f"Calculator result: {result}")
#         ])
#         return {"messages": new_messages, "next": "agent"}
    
#     # No tool use, just continue
#     return {"messages": messages + [response], "next": END}


def run_agent(state: AgentState) -> AgentState:
    messages = state["messages"]
    print("Current messages:", messages)  # Debug logging
    response = agent.invoke({"messages": messages})
    print("Agent response:", response)    # Debug logging
    
    content = response.content
    if "ACTION: calculator" in content:
        expression = content.split("INPUT:")[1].strip()
        print("Calculator expression:", expression)  # Debug logging
        result = calculator(expression)
        print("Calculator result:", result)         # Debug logging
        new_messages = list(messages)
        new_messages.extend([
            AIMessage(content=content),
            HumanMessage(content=f"Calculator result: {result}")
        ])
        return {"messages": new_messages, "next": "agent"}
    
    return {"messages": messages + [response], "next": END}

def format_output(state: AgentState) -> dict:
    """Format the final output from the messages"""
    messages = state["messages"]
    return {"output": messages[-1].content}

workflow = Graph()
workflow.add_node("agent", run_agent)
workflow.add_node("output", format_output)
workflow.set_entry_point("agent")
workflow.add_edge("agent", "output")

workflow.set_finish_point("output")


In [33]:
# 5. Create the graph
# workflow = Graph()
# workflow.add_node("agent", run_agent)
# workflow.set_entry_point("agent")



In [34]:
# 6. Compile the graph
chain = workflow.compile()



In [35]:
result = chain.invoke({
    "messages": [HumanMessage(content="What is 123123 multiplied by 123123123?")]
})

Current messages: [HumanMessage(content='What is 123123 multiplied by 123123123?', additional_kwargs={}, response_metadata={})]
Agent response: content='ACTION: calculator\nINPUT: 123123 * 123123123' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 112, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-adff2a63-286c-4fcc-8275-9d0743e1b959-0' usage_metadata={'input_tokens': 112, 'output_tokens': 15, 'total_tokens': 127, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Calculator expression: 123123 * 123123123
Calculator result: 15159288273129


In [36]:
type(result)

dict

In [37]:
result

{'output': 'Calculator result: 15159288273129'}

In [39]:
123123*123123123==15159288273129

True